# A. 代码内容说明
（启动训练及推理的方式、必要注释等）

项目文件为**main.ipynb**
直接运行所有代码块，完成训练与测试，如图：![](https://ai-studio-static-online.cdn.bcebos.com/e349720a7bb2432e92e4ce1ed54818d5859a89bdae6e4e2ba029fa2502c761c5)

运行结束后即得到测试数据预测文件： **CVPR_2022_NAS_Track2_submit_A_frnn.json**

若要直接读取已经训练好的模型，并在测试集进行预测，则直接运行**main_test.ipynb**

文件的各代码块有必要的注释。

对于各个任务，最优模型参数保存在**checkpoint/taskx_best.pdparams**文件，x为0,1，...，7。顺序为基线给定的顺序。**checkpoint_bkp**文件夹中有备份。

# B. 模型构建思路及调优过程
（可具体包括：思路框架图、思路步骤详述、模型应用+调优过程）

主要是想利用模糊粗糙神经网络作为预测模型

模型包括：
1. 模糊化层
2. 模糊规则层
3. 粗糙层
4. 输出层

思路步骤详述：
1. 模糊粗糙神经网络能够将输入模糊化，转化为模糊隶属度，之后得到模糊规则，粗糙层得到模糊粗糙隶属度，最后得到输出。
2. 模糊粗糙神经网络可通过对网络架构输入进行学习，从而对网络精度进行预测。

模型应用+调优过程：
1. batch_size：大的batch_size性能不好，设为1后，发现预测性能明显好于基线
2. learning_rate：一般，固定的学习率效果不好，采用了cosine退火
3. 减少网络层：去除粗糙层、模糊层后，感觉效果有提升，只简单地比较了第一个task
4. 增加隶属函数数目：FRNN的num_mf参数由2增到16
5. 融合模糊化与全连接：效果没提升，没采用
6. 保存最优模型：调试中发现，模型性能不稳定，不能保证最后得到的是最优的，因此，在优化过程中，每个epoch后，计算在验证集上的Kendalltau指标值，以保存最优模型参数。


# CVPR 2022 NAS workshop Track 2 Demo

为了解决神经网络结构搜索的搜索效率和效果问题，百度联合悉尼科技大学和美国北卡罗来纳大学，以轻量级神经网络结构搜索技术为研究课题，提供技术、数据、算力等支持，全球招募挑战者共同切磋交流，打造最前沿的AI模型结构搜索技术。 百度在NAS领域原创性了提出了GP-NAS（CVPR），SA-NAS（IJCV）等AutoDL算法，使用研发的AutoDL算法先后7次CVPR，ECCV等国际比赛夺得世界第一，提交了200余项中国/美国专利申请。并孵化了[PaddleSlim](https://github.com/PaddlePaddle/PaddleSlim)模型压缩平台，PaddleSlim是百度基于飞桨PaddlePaddle打造的开源模型压缩工具库，囊括了深度学习模型压缩领域常用的量化、剪枝、蒸馏、模型结构搜索等方法，并且打造了CV和NLP领域的模型压缩方案。
   
本次比赛分为两个赛道，赛道一为[超网络赛道](https://aistudio.baidu.com/aistudio/competition/detail/149/0/introduction)，旨在解决OneshotNAS的一致性问题；赛道二为[模型性能预测赛道](https://aistudio.baidu.com/aistudio/competition/detail/150/0/introduction)，旨在不做任何训练的情况，准确的预测任意模型结构在特定评测集的性能。**获胜的队伍会被邀请在[CVPR 2022 NAS workshop](https://www.cvpr-nas.com/)上宣讲队伍的技术方案。 此外，各Track 前三名会被邀请提交论文（extended abstract论文可以不通过cmt系统提交，regular论文需要通过cmt系统提交) ，论文要求详见[CVPR NAS workshop论文提交页面](https://www.cvpr-nas.com/Paper_Submission)**

# 赛题介绍
在不训练的情况下，准确的预测任意模型结构性能非常重要。基于此，我们不仅可以深度的分析怎样的模型结构会有很好的性能，怎样的模型性能会很差。同时还能够预测出满足任意硬件延时约束下的最优的模型结构。本赛事提供了部分（小样本）模型结构与模型精度之间对应关系的bench mark,参赛选手既可以通过黑盒的方式直接进行训练，也可以使用白盒的方式进行参数估计。最终预测泛化性能最好的模型将夺得冠军。相比去年的赛题，今年要求选手预测[VIMER-UFO](https://github.com/PaddlePaddle/VIMER/tree/main/UFO)中，任意单模型在多个任务上的性能。UFO这个技术设想的出发点是视觉的大一统，即一个模型能够覆盖所有主流的视觉任务。从垂类应用出发，选择了人脸、人体、车辆、商品四个任务作为视觉模型大一统的第一步。

在本赛道中，超网络基于ViT-Base构建搜索空间，搜索空间维度有网络深度（depth）、自注意力头的数目（num_heads）、前向计算网络的膨胀系数（mlp_ratio)，其变化范围为depth \in {10, 11, 12}，num_heads \in {10, 11, 12}，mlp_ratio \in {3.0, 3.5, 4.0}，搜索空间中有 9^10 + 9^11 + 9^12 个不同的子网络。
子网的编码的长度为37，包括1位depth编码，以及12组的3位编码，分别指示为num_heads、mlp_ratio和embed_dim（在本次赛题中embed_dim为768，不作为搜索维度），实际depth小于12，则后尾填充0。对于depth编码，'j'，'k'和'l'分别表示10，11和12；对于num_heads编码，'1'，'2'和'3'表示12，11和10；对于mlp_ratio编码，'1'，'2'和'3'表示4.0, 3.5, 3.0，对于embed_dim编码，'1'表示768。以j111231321311311221231121111231000000为例，子网结构的depth为10，10层模型的num_heads的列表为[12, 11, 10, 10, 10, 11, 11, 12, 12, 11]，mlp_ratio的列表为[4.0, 3, 3.5, 4.0, 4.0, 3.5, 3, 3.5, 4.0, 3]，embed_dim的列表为[768, 768, 768, 768, 768, 768, 768, 768, 768, 768]。

我们提供了训练数据，包括500个样本，其中输入为模型结构，标签为每个结构在8个任务上的相对排序rank，rank取值为0到499的整数；测试数据包括99500个样本，选手需要根据样本的结构信息训练多任务预测器（可以每个任务单独训练，也可以联合训练），并预测测试数据的99500个结构在8个任务上的排序，取值范围为0到99499。提交格式见『提交结果』

比赛分A/B榜单，A/B榜单都基于选手提交的同一份提交文件，但是计算分数的节点的编号不同。比赛提交截止日期前仅A榜对选手可见，比赛结束后B榜会对选手公布，比赛最终排名按照选手成绩在B榜的排名。

A榜队伍历史最高成绩（的json文件）和最后一次提交的json文件，会被用于计算Ｂ榜成绩，两个结果取最优。  

# 训练集说明

In [4]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data
!ls /home/aistudio/data/data134077

data134077
CVPR_2022_NAS_Track1_test.json	CVPR_2022_NAS_Track2_train.json
CVPR_2022_NAS_Track2_test.json	resnet48.pdparams


In [18]:
# 读取训练数据, 训练集包含500个模型结构，以及这些结构在cplfw，market1501，dukemtmc等8个任务上的性能排序
import json
with open('/home/aistudio/data/data134077/CVPR_2022_NAS_Track2_train.json', 'r') as f:
    train_data = json.load(f)
print(train_data['arch1'])
print('train_num:',len(train_data.keys()))

{'cplfw_rank': 9, 'market1501_rank': 361, 'dukemtmc_rank': 426, 'msmt17_rank': 433, 'veri_rank': 327, 'vehicleid_rank': 480, 'veriwild_rank': 425, 'sop_rank': 367, 'arch': 'l231131331121121331111211121331321321'}
train_num: 500


# 处理训练数据


In [6]:
def convert_X(arch_str):
    temp_arch = []
    for elm in arch_str:
        if elm == 'l':
            temp_arch.append(1)
        elif elm == 'j':
            temp_arch.append(2)
        elif elm == 'k':
            temp_arch.append(3)
        else: 
            temp_arch.append(int(elm))
    return(temp_arch)
train_list = [[],[],[],[],[],[],[],[]]
arch_list_train = []
name_list = ['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']
for key in train_data.keys():
    for idx, name in enumerate(name_list):
        train_list[idx].append(train_data[key][name])
    arch_list_train.append(convert_X(train_data[key]['arch']))
print(arch_list_train[0])

[1, 2, 3, 1, 1, 3, 1, 3, 3, 1, 1, 2, 1, 1, 2, 1, 3, 3, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 3, 3, 1, 3, 2, 1, 3, 2, 1]


# 训练各任务预测器

In [19]:
# 安装paddleslim
!pip install paddleslim

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.


In [20]:
# 加载python包
import os
import numpy as np
from paddle.io import Dataset,DataLoader
from paddle.vision import transforms as T
from paddle.nn import functional as F
import cv2
import paddle
import matplotlib.pyplot as plt
import paddle.nn as nn
from tqdm import tqdm
import math
from math import sqrt
import time

import pdb

In [21]:
# 记录并计算平均值
class AverageMeter(object):
    """
    Computes and stores the average and current value
    Copied from: https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [22]:
# 模糊化层
# 基于模糊集理论，对于每个输入，通过若干个隶属函数，转化为隶属度（0~1）
class Fuzzify(nn.Layer):
    def __init__(self,dim=2,flag_fix=False): # dim为每个输入对应的隶属函数数目，隶属函数选用高斯函数
        super(Fuzzify, self).__init__()
        self.dim = dim
        if not flag_fix: # 隶属函数的参数是否是固定的
            param_c = self.create_parameter([1,dim]) # 高斯函数的期望值
            self.add_parameter("param_c", param_c)
            param_d = self.create_parameter([1,dim]) # 高斯函数的标准差
            self.add_parameter("param_d", param_d)
            # init
            self.param_c.set_value(np.random.rand(1,dim).astype("float32")*3) # 初始化[0,3)，因为每个输入值取值为{0,1,2,3}
            self.param_d.set_value(np.ones((1,dim)).astype("float32")) # 初始化
        else:
            self.param_c = paddle.zeros([1, dim])
            #self.param_d = paddle.ones([1, dim])
            param_d = self.create_parameter([1,dim])
            self.add_parameter("param_d", param_d)
            # init
            for _ in range(dim):
                self.param_c[0, _] = _
            self.param_d.set_value(np.ones((1,dim)).astype("float32"))

    def forward(self,x):
        x_out = []
        for i in range(self.dim):
            x_out.append(paddle.exp(-(x - self.param_c[0, i]) ** 2 * (2 * self.param_d[0, i] ** 2))) # 除以2\delta^2 改为 乘以2\delta^2
        return paddle.concat(x_out, 1)

In [23]:
class FRNN(nn.Layer): # 模糊粗糙神经网络
    def __init__(self,num_feat,num_mf=2,num_fr=7,num_rgh=3,num_classes=1):
        super(FRNN, self).__init__()
        '''
        FRNN: fuzzification, fuzzy rule, rough, output
        '''

        '''
        self.fuzz = Fuzzify(num_mf,flag_fix=False)
        self.cnsq = nn.Sequential(
            nn.Linear(num_feat,num_feat*num_mf),
            nn.BatchNorm1D(num_feat*num_mf),
            nn.ReLU(),
        )
        self.out  = nn.Sequential(
            nn.Linear(2*num_feat*num_mf,num_classes),
            nn.Sigmoid()
        )
        '''
        self.frnn = nn.Sequential(
            Fuzzify(num_mf,flag_fix=False), # 模糊化层
            #nn.Linear(num_feat*num_mf,num_fr), # 模糊规则层
            #nn.BatchNorm1D(num_fr),
            #nn.ReLU(),
            #nn.Linear(num_fr,num_rgh), # 粗糙层
            #nn.BatchNorm1D(num_rgh),
            #nn.ReLU(),
            #nn.Linear(num_rgh,num_classes), # 输出层
            #nn.Linear(num_fr,num_classes), # 输出层
            nn.Linear(num_feat*num_mf,num_classes), # 输出层
            nn.Sigmoid() # 转到0~1
        )
        self.flag = False

    def forward(self,x):
        '''
        x1 = self.fuzz(x)
        x2 = self.cnsq(x)
        x  = self.out(paddle.concat(x=[x1, x2], axis=1))
        if not self.flag:
            print(x.shape, x)
            self.flag = True
        return x
        '''
        return self.frnn(x)

In [24]:
# 评估网络模型
def evaluation(model, params_file_path, valid_loader,flag_test=False):
    if params_file_path is not None:
        model_state_dict = paddle.load(params_file_path)
        model.load_dict(model_state_dict)
    model.eval()

    all_pd = []
    all_gt = []
    for idx, data in enumerate(valid_loader()):
        if flag_test:
            arch_v = data
        else:
            arch_v, score_label = data

        scores_predict = model(arch_v)

        all_pd.append(scores_predict)
        if not flag_test:
            all_gt.append(score_label)

    all_pd = paddle.concat(all_pd)
    if flag_test: # 测试集没groundtruth
        return all_pd.numpy()
    else:
        all_gt = paddle.concat(all_gt)
        return scipy.stats.stats.kendalltau(all_pd.numpy(), all_gt.numpy())

In [13]:
# 训练预测网络模型
def train(model,data_loader,eval_dataloader,task=0,Epoches=10,lr=0.05,eta_min=0.01):
    paddle.set_device('cpu') # 不必使用GPU

    # 启动训练
    model.train()
    # 获得数据读取器
    data_loader = data_loader
    nBatch = len(data_loader)
    # 使用SGD优化器，学习率使用cosine退火
    scheduler = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=lr, T_max=Epoches*nBatch, eta_min=eta_min, verbose=False)
    opt = paddle.optimizer.SGD(learning_rate=scheduler, parameters=model.parameters())
    # opt = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters())

    losses = AverageMeter()
    data_time = AverageMeter()

    cor_valid = 0

    for epoch in range(0, Epoches):
        with tqdm(total=nBatch//50+1,
                  desc='Train Epoch #{}'.format(epoch + 1)) as t:
            end = time.time()
            for idx, data in enumerate(data_loader()):
                data_time.update(time.time() - end)
                new_lr = scheduler.last_lr
                # 获得数据
                arch_v, scores_label = data
                # 计算出算法的前向计算结果
                scores_predict = model(arch_v)
                # 计算loss
                loss = F.square_error_cost(scores_predict, scores_label)
                avg_loss = paddle.mean(loss)

                # 损失函数下降，并清除梯度
                avg_loss.backward()
                opt.step()
                opt.clear_grad()
                scheduler.step()

                # measure accuracy and record loss
                if np.isnan(avg_loss.item()):
                    print(avg_loss.item())
                losses.update(avg_loss.item(), arch_v.shape[0])

                if idx % 50 == 0 or idx == nBatch - 1:
                    t.set_postfix({
                        'loss': losses.avg,
                        'lr': new_lr,
                        'data_time': data_time.avg,
                    })
                    t.update(1)
                end = time.time()

        # 每个epoch 保存一次模型
        paddle.save(model.state_dict(), './checkpoint/task' + str(task) + '_epoch' + str(epoch) + '.pdparams')

        tmp = evaluation(model, None, eval_dataloader)
        print('Kendalltau:', tmp)
        # 保存在验证集上表现最好的模型
        if tmp.correlation > cor_valid:
            paddle.save(model.state_dict(), './checkpoint/task' + str(task) + '_best' + '.pdparams')
            cor_valid = tmp.correlation


In [14]:
# 构建数据集
class ArchDataset(Dataset):
    def __init__(self, data, label, transform=None):
        super(ArchDataset, self).__init__()
        self.data = data
        self.label = label
        self.transform = transform

    def __getitem__(self, index):
        if self.label is not None:
            return np.array(self.data[index]).astype("float32"), np.array(self.label[index]).astype("float32")
        else:
            return np.array(self.data[index]).astype("float32")

    def __len__(self):
        return len(self.data)

In [25]:
from paddleslim.nas import GPNAS
import numpy as np
import scipy
import scipy.stats

gp_list = []
frnn_list = []

for i in range(len(train_list[:])):
    # 每个任务有该任务专属的gpnas、FRNN预测器
    gp_list.append(GPNAS(2,2))
    frnn_list.append(FRNN(num_feat=37,num_mf=16,num_fr=8,num_rgh=4,num_classes=1))

train_num = 400
batch_size = 1
Epoches = 100


for i in range(len(train_list[:])):
    # 划分训练及测试集
    X_all_k, Y_all_k  = np.array(arch_list_train), np.array(train_list[i])
    X_train_k, Y_train_k, X_test_k, Y_test_k = X_all_k[0:train_num:1], Y_all_k[0:train_num:1], X_all_k[train_num::1], Y_all_k[train_num::1]
    # 获取数据读取器
    Y_train_k = Y_train_k / max(Y_train_k)
    train_dataset = ArchDataset(X_train_k, Y_train_k)
    eval_dataset = ArchDataset(X_test_k, Y_test_k)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
    eval_dataloader = DataLoader(eval_dataset, batch_size=100, shuffle=True, drop_last=False)
    # 观察数据读取是否正常
    data, label = next(train_dataloader())
    #print([data[0], label[0]])
    #
    train(frnn_list[i], train_dataloader,eval_dataloader,i,Epoches=Epoches,lr=0.05,eta_min=0.002)
    # 读取最优模型参数
    Kendalltau = evaluation(frnn_list[i], './checkpoint/task' + str(i) + '_best' + '.pdparams', eval_dataloader)
    print('Kendalltau:', Kendalltau)
    # 初始该任务的gpnas预测器参数
    gp_list[i].get_initial_mean(X_train_k[0::2],Y_train_k[0::2])
    init_cov = gp_list[i].get_initial_cov(X_train_k)
    # 更新（训练）gpnas预测器超参数
    gp_list[i].get_posterior_mean(X_train_k[1::2],Y_train_k[1::2])  
   
    # 基于测试评估预测误差   
    error_list_gp = np.array(Y_test_k.reshape(len(Y_test_k),1)-gp_list[i].get_predict(X_test_k))
    error_list_gp_j = np.array(Y_test_k.reshape(len(Y_test_k),1)-gp_list[i].get_predict_jiont(X_test_k, X_train_k[::1], Y_train_k[::1]))
    print('AVE mean gp :',np.mean(abs(np.divide(error_list_gp,Y_test_k.reshape(len(Y_test_k),1) ))))
    print('AVE mean gp jonit :',np.mean(abs(np.divide(error_list_gp_j,Y_test_k.reshape(len(Y_test_k),1) ))))
    y_predict = gp_list[i].get_predict_jiont(X_test_k, X_train_k[::1], Y_train_k[::1])

    #基于测试集评估预测的Kendalltau
    print('Kendalltau:',scipy.stats.stats.kendalltau( y_predict,Y_test_k))

[05-21 11:27:54 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddleslim/common/analyze_helper.py:22: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://matplotlib_inline.backend_inline' by the following code:
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/runpy.py", line 85, in _run_code
    exe

KeyboardInterrupt: 

# 查看测试集

In [ ]:
with open('/home/aistudio/data/data134077/CVPR_2022_NAS_Track2_test.json', 'r') as f:
    test_data = json.load(f)
test_data['arch99997']

# 处理测试集数据

In [ ]:
test_arch_list = []
for key in test_data.keys():
    test_arch =  convert_X(test_data[key]['arch'])
    test_arch_list.append(test_arch)
print(test_arch_list[99499])

# 预测各任务上的测试集的结果

In [ ]:
batch_size = 512
rank_all = []
pd_all_frnn = [] # 预测值
rank_all_frnn = [] # 预测排序
for task in range(len(name_list)):
    print('Predict the rank of:', name_list[task])
    
    X_test_k = np.array(test_arch_list)
    test_dataset = ArchDataset(X_test_k, None)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    # slow mode
    # rank_all.append(gp_list[task].get_predict_jiont(np.array(test_arch_list), np.array(arch_list_train), np.array(train_list[task])))
    # fast mode
    rank_all.append(gp_list[task].get_predict(np.array(test_arch_list)))
    # 测试最优模型参数
    all_pd = evaluation(frnn_list[task], './checkpoint/task' + str(task) + '_best' + '.pdparams',
                        test_dataloader, flag_test=True)
    pd_all_frnn.append(all_pd)
    tmp = np.argsort(all_pd.reshape(-1))
    tmp2 = np.argsort(tmp) # 获得排名
    rank_all_frnn.append(tmp2)

 # 生成提交结果

In [ ]:
for idx,key in enumerate(test_data.keys()):
    test_data[key]['cplfw_rank'] = int(rank_all[0][idx][0])
    test_data[key]['market1501_rank'] = int(rank_all[1][idx][0])
    test_data[key]['dukemtmc_rank'] = int(rank_all[2][idx][0])
    test_data[key]['msmt17_rank'] = int(rank_all[3][idx][0])
    test_data[key]['veri_rank'] = int(rank_all[4][idx][0])
    test_data[key]['vehicleid_rank'] = int(rank_all[5][idx][0])
    test_data[key]['veriwild_rank'] = int(rank_all[6][idx][0])
    test_data[key]['sop_rank'] = int(rank_all[7][idx][0])
print('Ready to save results!')
with open('./CVPR_2022_NAS_Track2_submit_A.json', 'w') as f: # 得到baseline
    json.dump(test_data, f)

In [ ]:
for idx,key in enumerate(test_data.keys()):
    test_data[key]['cplfw_rank'] = int(rank_all_frnn[0][idx])
    test_data[key]['market1501_rank'] = int(rank_all_frnn[1][idx])
    test_data[key]['dukemtmc_rank'] = int(rank_all_frnn[2][idx])
    test_data[key]['msmt17_rank'] = int(rank_all_frnn[3][idx])
    test_data[key]['veri_rank'] = int(rank_all_frnn[4][idx])
    test_data[key]['vehicleid_rank'] = int(rank_all_frnn[5][idx])
    test_data[key]['veriwild_rank'] = int(rank_all_frnn[6][idx])
    test_data[key]['sop_rank'] = int(rank_all_frnn[7][idx])
print('Ready to save results!')
with open('./CVPR_2022_NAS_Track2_submit_A_frnn.json', 'w') as f: # 得到提出方案的预测值
    json.dump(test_data, f)


In [27]:
!pip install pipreqs
!pipreqs --use-local --encoding=utf8

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=74c5da908539b8d18f92bf712b07930b23a9e2a13ddd3107a368392aef3d65eb
  Stored in directory: /home/aistudio/.cache/pip/wheels/17/d9/6e/6b3eb21ad36a388d8d7eb1c1c61e34c627876458e6fc37790d
Successfully built docopt
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
INFO: Successfully saved requirements file in /home/aistudio/requirements.txt


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 